In [3]:
import pandas as pd
import numpy as np
import datetime

import tensorflow as tf
import cv2     # for capturing videos

import shutil
import streamlit as st
import tempfile
import warnings
import os
from PIL import Image

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import decode_predictions
import numpy as np

import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline

videoFile = '/content/drive/MyDrive/tonick/vid_cars.mp4'

# Build Model

In [4]:
model = VGG16(weights='imagenet', include_top=True)

In [5]:
def identify_frames(img_path):    
  img = image.load_img(img_path, color_mode='rgb', target_size=(224, 224))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  features = model.predict(x)
  return features

# Video Processing

## Frame Exctracion

In [6]:
def extract_frames(vid_path):
    # set video file path of input video with name and extension
    vid = cv2.VideoCapture(vid_path)


    if not os.path.exists('images'):
        os.makedirs('images')
    else:
        shutil.rmtree('images')
        os.makedirs('images')
    #identify frame
    index = 0
    while(True):
        

        # Extract images
        ret, frame = vid.read()
        # end of frames
        if not ret: 
            break
        # Saves images
        name = 'frame' + str(index) + '.jpg'
        print ('Creating... ' + name)
        cv2.imwrite(name, frame)

        # next frame
        index += 1
    vid.release()

In [7]:
# extract_frames(videoFile)

## Frame Encoding

In [8]:
def label_frames(inputpath, outputpath, videoFile):

    if not os.path.exists('encoded_images'):
        os.makedirs('encoded_images')
    else:
        shutil.rmtree('encoded_images')
        os.makedirs('encoded_images')

    count = 0
    cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    font_scale = 3
    font = cv2.FONT_HERSHEY_PLAIN
    obj_detected = []


    while(cap.isOpened()):
        ret, frame = cap.read()
        if (ret != True):
            break
        filename =inputpath+"/frame"+str(count)+".jpg"
        p = decode_predictions(identify_frames(filename))
        label = p[0][0][1]
        if label not in obj_detected:
            obj_detected.append(label)
        cv2.putText(frame, "[*Object Detected] : "+str(label), (25, 60), font, fontScale=font_scale,color=(0,255,0), thickness=3)
        cv2.imwrite(outputpath+ str(label) + " " + str(count)+".jpg", frame)
        print ('Econded... ' + str(label) + "... " + str(count))
        count += 1
    cap.release()
    print ("[**Frame Encoding] - Done")

## Video Regeneration

In [9]:
def build_video(inputpath,outputpath,fps):
    if not os.path.exists('videos'):
        os.makedirs('videos')
    else:
        shutil.rmtree('videos')
        os.makedirs('videos')

    image_array = []
    files = [f for f in os.listdir(inputpath) if os.path.isfile(os.path.join(inputpath, f))]
    files.sort(key = lambda x: int(x.split()[1][:-4]))
    for i in range(len(files)):
        img = cv2.imread(inputpath + files[i])
        size =  (img.shape[1],img.shape[0])
        img = cv2.resize(img,size)
        image_array.append(img)
    fourcc = cv2.VideoWriter_fourcc(*'H264')
    out = cv2.VideoWriter(outputpath,fourcc, fps, size)
    for i in range(len(image_array)):
        out.write(image_array[i])
    out.release()


# Get Objects Detected

In [11]:
files = [f for f in os.listdir("./encoded_images") if os.path.isfile(os.path.join("./encoded_images", f))]
fi = [i.split()[0] for i in files] 
detected_obj = list(set(fi))

# Main Menu Options

## User Input Dashboard

In [12]:
%%writefile user.py
import streamlit as st
import tempfile
from base import *
from model import *
from video import *

def filter_frame(txt_search):
    files = [f for f in os.listdir("./encoded_images") if os.path.isfile(os.path.join("./encoded_images", f))]
    files.sort(key = lambda x: int(x.split()[1][:-4]))
    indices = [i for i, s in enumerate(files) if txt_search.lower() in s]
    return files[indices[0]]

files = [f for f in os.listdir("./encoded_images") if os.path.isfile(os.path.join("./encoded_images", f))]
fi = [i.split()[0] for i in files] 
detected_obj = list(set(fi))

def app():

    global rez

    rez = "..."


    f = st.file_uploader("Upload video")

    s1, s2, s3 = st.beta_columns([1,4,2])

    with s2:
        search = st.text_input("Search for objects in video")
        if search in detected_obj:
            rez = "Found"
        else:
            rez = "Not Found"
        
    with s3:
        st.write('Search results')
        if f is None:
            status = st.write(f'{search} ...')
        else:
            status = st.write(f'{search} {rez}')


    with st.beta_container():

        col1, col2 = st.beta_columns([2,6])

        with col1:
            st.write("Objects in video")
            for i in range(len(detected_obj)):
                st.write(str(i+1) +  " "+str(detected_obj[i]))
        with col2:

            if f is not None:
                tfile = tempfile.NamedTemporaryFile(delete=False) 
                tfile.write(f.read())

                with st.spinner('[*Extracting Frames] - Process Initialized...'):
                    extract_frames(tfile.name, "./images")
                st.success('[*Extracting Frames] - Process Completed...')

                with st.spinner('[**Encoding Frames] - Process Initialized...'):
                    label_frames(inputpath="./images", outputpath='./encoded_images/', videoFile=r"C:\Users\Blessed\Videos\vid_cars.mp4")
                st.success('[*Encoding Frames] - Process Completed...')

                with st.spinner('[***Building Frames] - Process Initialized...'):
                    build_video(inputpath='./encoded_images/', outputpath='./videos/video.mp4',fps=5)
                st.success('[*Building Frames] - Process Completed...')

                if search.lower() in detected_obj:
                    rez = "found"
                    img = Image.open('./encoded_images/'+str(filter_frame(search)))
                    st.image(img, caption=search.lower() + " frame found")
                else:
                    play_video('./videos/video.mp4')

Writing user.py


## Home Dashboard with Preloaded Video

In [13]:
%%writefile home.py
import streamlit as st
from base import *

files = [f for f in os.listdir("./en_images") if os.path.isfile(os.path.join("./en_images", f))]
fi = [i.split()[0] for i in files] 
detected_obj = list(set(fi))

def filter_frame(txt_search):
    files = [f for f in os.listdir("./en_images") if os.path.isfile(os.path.join("./en_images", f))]
    files.sort(key = lambda x: int(x.split()[1][:-4]))
    indices = [i for i, s in enumerate(files) if txt_search.lower() in s]
    return files[indices[0]]

def app():
    global h_rez

    h_rez = "..."

    st.title('Object(s) Detection in video')

    hs1, hs2, hs3 = st.beta_columns([1,4,2])

    with hs2:
        h_search = st.text_input("Search for objects in video")
        if h_search.lower() in detected_obj:
            h_rez = "found"
        else:
            h_rez = "not found"
        
    with hs3:
        st.write('Search results')
        if h_search.lower() is None:
            status = st.write(f'{h_search} ...')
        else:
            status = st.write(f'{h_search} {h_rez}')


    if len(detected_obj) > 6:
        col1, col2, col3 = st.beta_columns([2,2,6])
        with col1:
            st.subheader("Objects in video")
            for i in range(0,7):
                st.write(str(i+1) +  " "+str(detected_obj[i]))
        with col2:
            st.write("")
            st.write("")
            st.write("")
            st.write("")
            for i in range(7,len(detected_obj)-1):
                st.write(str(i+1) +  " "+str(detected_obj[i]))
        with col3:
            st.write("")
            st.write("")
            st.write("")
            st.write("")
            if h_search.lower() in detected_obj:
                h_rez = "found"
                img = Image.open('./en_images/'+str(filter_frame(h_search)))
                st.image(img, caption=h_search.lower() + " frame found")
            else:
                play_video("video.mp4")
    else:
        col1, col2 = st,beta_columns([2,6])
        with col1:  
            st.write("Objects in video")
            for i in range(len(detected_obj)):
                st.write(str(i+1) +  " "+str(detected_obj[i]))
        with col2:
            play_video("video.mp4")

Writing home.py


## About Page

In [14]:
%%writefile about.py
import streamlit as st

def app():
    
    st.title('Object Detection using VGG16 Model...')

    c1, c2, c3 = st.beta_columns([4,2,4])
    with c2:
        st.write('Done by:')

    b1,m1,l1 = st.beta_columns([3,1,3])
    with b1:
        st.write('Blessed Mutengwa - R182565F')
        st.write('blessedmutengwa@gmail.com')
    with m1:
        st.write("|")
    with l1:
        st.write('Linval Chisoko - R182565F')
        st.write('linvaltchisoko@gmail.com')

Writing about.py


## Base File

In [15]:
%%writefile base.py
import streamlit as st
import os
from PIL import Image


def play_video(vid_path):
    try:
        video_file = open(vid_path, 'rb')
        video_bytes = video_file.read()
        st.video(vid_path)
    except FileNotFoundError:
        pass

Writing base.py


## Streamlit Main App File

In [16]:
%%writefile app.py
import streamlit as st
st.set_page_config(layout="wide")

from multiapp import MultiApp
from apps import home, user, about # import your app modules here

app = MultiApp()

# Add all your application here
app.add_app("Home", home.app)
app.add_app("User Upload", user.app)
app.add_app("About Project", about.app)
# The main app
app.run()

Writing app.py
